In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tqdm.notebook import tqdm
from IPython.display import HTML

In [2]:
N = 15 # Number of cities

In [ ]:
c = np.random.rand(N, 2) # Generate random coordinates to each city

# --- Plot cities ---
for i in range(N):
    plt.scatter(c[i,0], c[i,1], label=f'City {i+1}')

plt.legend()
plt.grid()
plt.show()

In [4]:
# -- Distances ---
d = np.zeros(shape=(N,N))
for i in range(N):
    for j in range(N):
        d[i,j] = np.linalg.norm(c[i,:] - c[j,:]) # Euclidean distance between c_i and c_j

In [5]:
def energy(_P):
    # The energy is the total travel distance
    en = 0
    for i in range(N-1):
        en += d[_P[i], _P[i+1]]
    en += d[_P[0], _P[-1]]
    return en

def disturb(_P):
    # Swap two positions randomly
    i = np.random.randint(0, N-1)
    j = np.random.randint(i+1, N)
    _P = _P.copy()
    x = _P[j]
    _P[j] = _P[i]
    _P[i] = x
    return _P

def anneal(P_init, n_iterations, n_disturbances, n_success, alpha=0.9, update_f=None):
    # Simulated Annealing Function
    # Params:
    #   P_init (Numpy 1D array) - initial state
    #   n_iterations (int) - maximum number of annealing epochs
    #   n_disturbances (int) - maximum number of disturbances per epoch
    #   n_success (int) - maximum number of successful disturbances per epoch
    #   alpha (float) - temperature reduction factor
    #   update_f (callable) - callback function to call after each epoch
    _P = P_init.copy() # Current state

    temperature = 1000.0 # Current temperature
    
    for i in tqdm(range(n_iterations)): # Epochs
        success_count = 0
        
        for j in range(n_disturbances):
            _Pn = disturb(_P)
            delta_en = energy(_Pn) - energy(_P)
            
            if delta_en <= 0 or np.exp(-delta_en/temperature) > np.random.rand():
                _P = _Pn
                success_count += 1
            
            if success_count >= n_success:
                break
        
        temperature *= alpha

        if update_f is not None:
            update_f(i, temperature, _P)

        if success_count == 0: #Stuck in local minina
            break
    
    return _P

In [ ]:
P_init = np.arange(N) # Create an initial state as [1, 2, ..., N]
np.random.shuffle(P_init) # Shuffle P_init get a random initial state

P_list = [P_init] # List of state evolution
T_list = [1000.0] # List of temperature evolution
E_list = [energy(P_init)] # List of energy evolution

def append_to_list(i, _T, _P): # Callback function
    # Params:
    #   i (int) - current epoch
    #   _T (float) - current temperature
    #   _P (Numpy 1D array) - current state
    P_list.append(_P)
    T_list.append(_T)
    E_list.append(energy(_P))

P_star = anneal(P_init, 1000, 100, 5, update_f=append_to_list)
P_star

In [ ]:
# --- Calculation of total travel distance ---
print('Total distance:', float(energy(P_star)))

In [ ]:
# --- Solution plot ---
for i in range(N-1):
    plt.plot(c[P_star[i:i+2],0], c[P_star[i:i+2],1], color='gray', linestyle='dashed')

plt.plot(c[[P_star[0], P_star[N-1]], 0], c[[P_star[0], P_star[N-1]], 1], color='gray', linestyle='dashed')

for i in range(N):
    plt.scatter(c[P_star[i],0], c[P_star[i],1], label=f'$P_{{{i+1}}}$')

plt.legend()
plt.grid()
plt.show()

In [9]:
# --- Animate the annealing proccess ---

fig, ax = plt.subplots(ncols=3, figsize=(10,4), constrained_layout=True)
fig.suptitle('Solving the Traveling Salesman Problem with Simulated Annealing (by Filipe)')

# --- Solution plot ---
ax[0].grid()
ax[0].set_title('Solution')
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')

lines = [ax[0].plot([], [], color='gray', linestyle='dashed', zorder=1)[0] for i in range(N+1)]

for i in range(N): # Plot a dot for each city
    ax[0].scatter(c[i,0], c[i,1], zorder=2)

# --- Temperature plot ---
ax[1].set_title('Annealing temperature')
ax[1].set_xlabel('Time')
ax[1].set_ylabel('Temp (ºC)')
ax[1].plot(np.arange(len(T_list))/len(T_list), T_list) # Plot the temperature curve

# Current temperature markers
temp_v_line, = ax[1].plot([0, 0], [np.min(T_list), np.max(T_list)], color='gray', linestyle='dashed', zorder=1)
temp_h_line, = ax[1].plot([0, 1], [0, 0], color='gray', linestyle='dashed', zorder=1)
temp_point = ax[1].scatter([0], [0], color='red', zorder=2)

# --- Energy plot ---
ax[2].set_title('Solution energy')
ax[2].set_xlabel('Time')
ax[2].set_ylabel('$Energy (J)$')
ax[2].set_ylim(np.min(E_list), np.max(E_list))
ax[2].set_xlim(0, 1)

# Energy evolution curve
energy_line, = ax[2].plot([], []) # Starts empty

# Current energy markers
cen_v_line, = ax[2].plot([0, 0], [np.min(E_list), np.max(E_list)], color='gray', linestyle='dashed', zorder=1)
cen_h_line, = ax[2].plot([0, 1], [0, 0], color='gray', linestyle='dashed', zorder=1)
cen_point = ax[2].scatter([0], [0], color='red', zorder=2)

# --- Animation ---

def animate(k):
    ax[0].set_title(f'Solution\n{k}-th epoch')
    
    # Plot the path found in the k-th epoch
    _P = P_list[k]
    for i in range(len(lines)):
        lines[i].set_data(c[_P[i:i+2],0], c[_P[i:i+2],1])
    lines[N].set_data(c[[_P[0], _P[N-1]], 0], c[[_P[0], _P[N-1]], 1])

    # Plot the temperature in the k-th epoch
    _T = T_list[k]
    temp_v_line.set_data([k/len(T_list), k/len(T_list)], [np.min(T_list), np.max(T_list)])
    temp_h_line.set_data([0, 1], [_T, _T])
    temp_point.set_offsets([k/len(T_list), _T])

    # Plot the energy evolution
    _E = E_list[k]
    energy_line.set_data(np.arange(k+1)/len(E_list), np.array(E_list[:k+1]))
    cen_v_line.set_data([k/len(E_list), k/len(E_list)], [np.min(T_list), np.max(T_list)])
    cen_h_line.set_data([0, 1], [_E, _E])
    cen_point.set_offsets([k/len(E_list), _E])

ani = FuncAnimation(fig, animate, frames=len(P_list))
plt.close()
HTML(ani.to_jshtml())

In [ ]:
ani.save('tsp_sa_filipe.mp4')
ani.save('tsp_sa_filipe.gif')